# 02 — EDA avançado: distribuições, grupos e storytelling

Objetivo: sair de gráficos soltos e montar uma EDA que gera comparações e insights reutilizáveis.

Tempo: ~25–30 min

## O que você vai fazer aqui

1- Entender **forma da distribuição** (cauda, assimetria, log-scale)  
2- Comparar **grupos** (categoria, região, segmento)  
3- Criar um “mini framework” de **insights replicáveis** (tabelas + gráficos)  

Dataset: `dados/sample/sales.csv` + `dados/sample/customers.csv`

In [ ]:
from __future__ import annotations

from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

def find_repo_root(start: Path | None = None) -> Path:
    """Sobe diretórios até achar README.md + pasta data."""
    cur = (start or Path.cwd()).resolve()
    for _ in range(10):
        if (cur / "README.md").exists() and (cur / "data").exists():
            return cur
        cur = cur.parent
    return Path.cwd().resolve()

root = find_repo_root()
DATA = root / "data"

In [ ]:
sales = pd.read_csv(DATA / "sample" / "sales.csv")
customers = pd.read_csv(DATA / "sample" / "customers.csv")

sales["date"] = pd.to_datetime(sales["date"])
customers["signup_date"] = pd.to_datetime(customers["signup_date"])

df = sales.merge(customers, on="customer_id", how="left")

df.head()

## Checagens rápidas (sanidade)

In [ ]:
# Tipos / nulos / duplicados
display(df.dtypes)
display(df.isna().mean().sort_values(ascending=False).head(10))

dups = df.duplicated("order_id").sum()
print("Duplicados por order_id:", dups)

## Distribuição de receita (revenue)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
ax.hist(df["revenue"], bins=30)
ax.set_title("Revenue - histograma")
ax.set_xlabel("revenue")
ax.set_ylabel("contagem")
plt.show()

In [ ]:
# Muitas vezes revenue tem cauda longa: experimente escala log
fig, ax = plt.subplots(figsize=(8, 4))
ax.hist(np.log1p(df["revenue"]), bins=30)
ax.set_title("log1p(revenue) - histograma")
ax.set_xlabel("log1p(revenue)")
ax.set_ylabel("contagem")
plt.show()

## Boxplot por categoria e segmento

In [ ]:
# Boxplot ajuda a comparar mediana e dispersão entre grupos
fig, ax = plt.subplots(figsize=(10, 4))
sns.boxplot(data=df, x="category", y="revenue", ax=ax)
ax.set_title("Revenue por categoria")
ax.set_xlabel("category")
ax.set_ylabel("revenue")
plt.xticks(rotation=20)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
sns.boxplot(data=df, x="segment", y="revenue", ax=ax)
ax.set_title("Revenue por segmento")
plt.show()

## Tabelas de apoio (top categorias, regiões e produtos)

In [ ]:
def top_table(data: pd.DataFrame, group_col: str, value_col: str = "revenue", n: int = 10) -> pd.DataFrame:
    return (
        data.groupby(group_col, as_index=False)[value_col]
        .sum()
        .sort_values(value_col, ascending=False)
        .head(n)
    )

display(top_table(df, "category", n=10))
display(top_table(df, "region", n=10))
display(top_table(df, "product", n=10))

## Série temporal (tendência mensal)

In [ ]:
monthly = (
    df.assign(month=df["date"].dt.to_period("M").dt.to_timestamp())
      .groupby("month", as_index=False)["revenue"].sum()
      .sort_values("month")
)

fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(monthly["month"], monthly["revenue"])
ax.set_title("Revenue total por mês")
ax.set_xlabel("mês")
ax.set_ylabel("revenue")
plt.xticks(rotation=30)
plt.show()

monthly.tail()

## Checklist rápido de insights (modelo simples)

In [ ]:
insights = []

# Exemplo 1: categoria líder
top_cat = df.groupby("category")["revenue"].sum().sort_values(ascending=False).head(1)
insights.append({
    "pergunta": "Qual categoria mais contribui para receita?",
    "insight": f"{top_cat.index[0]} lidera com {top_cat.iloc[0]:.2f} de receita."
})

# Exemplo 2: região líder
top_reg = df.groupby("region")["revenue"].sum().sort_values(ascending=False).head(1)
insights.append({
    "pergunta": "Qual região mais contribui para receita?",
    "insight": f"{top_reg.index[0]} lidera com {top_reg.iloc[0]:.2f} de receita."
})

insights_df = pd.DataFrame(insights)
display(insights_df)

# Exporta para markdown (bom para colar no README do projeto)
out_md = DATA / "output" / "insights_07_02.md"
out_md.parent.mkdir(parents=True, exist_ok=True)
out_md.write_text(insights_df.to_markdown(index=False), encoding="utf-8")
print("Salvo:", out_md)

## Exercícios (10–15 min)

1- Crie um gráfico de **revenue por dia** (linha).  
2- Faça um ranking de **clientes** por receita total e identifique o Top 5.  
3- Compare **ticket médio** por segmento (`revenue` / número de pedidos).